In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./K-Food_info.csv', encoding = 'cp949', index_col='음식명').drop('구분', axis = 1).fillna('No data')

In [3]:
data2 = pd.read_csv('./K-Food_Recipe.csv', encoding='cp949', index_col='RECIPE_NM_KO')

In [4]:
data3 = pd.read_csv('./recipe_resource.csv', encoding='cp949').drop(['IRDNT_SN', 'IRDNT_CPCTY', 'IRDNT_TY_CODE', 'IRDNT_TY_NM'], axis = 1)

In [5]:
Allergie = {'메밀': ['메밀', '메밀가루'],
            '밀': ['밀가루', '밀', '강력분', '박력분', '중력분', '빵', '부침가루', '바게트', '칼국수', '국수', '소면', '면', '쫄면', '라면'],
            '대두': ['두부', '어묵', '콩나물'],
            '호두': ['호두'],
            '땅콩': ['땅콩', '땅콩버터'],
            '복숭아': ['복숭아'],
            '게': ['꽃게', '참게', '대게', '킹크랩'],
            '고등어': ['고등어'],
            '토마토': ['토마토'],
            '돼지고기': ['돼지고기', '갈비', '다진돼지고기', '베이컨', '돼지고기육수', '삼겹살', '햄'],
            '계란': ['계란', '달걀'],
            '우유': ['우유', '치즈', '치즈가루', '모짜렐라치즈'],
            '닭고기': ['닭고기'],
            '쇠고기': ['쇠고기', '소고기', '갈비', '쇠고기육수', '양지머리', '안심', '등심'],
            '새우': ['새우', '대하', '건새우', '새우젓', '생새우'],
            '홍합': ['홍합'],
            '전복': ['전복'],
            '굴': ['굴', '굴소스'],
            '조개류': ['바지락', '꼬막', '골뱅이', '소라'],
            '오징어': ['오징어', '진미채', '오징어젓갈'],
            '아황산': ['소시지', '소세지', '햄', '스팸', '육포','베이컨']}

# 음식정보

In [6]:
def food_en(name):
    failtext = '요리를 찾지 못했습니다.'
    try:
        food_name = name
        food_name_en = data.loc[name]['영문']
        food_info_en = data.loc[name]['음식설명']

        return food_name, food_name_en, food_info_en
    except:
        food_name = name
        food_name_en = failtext
        food_info_en = failtext
        return food_name, food_name_en, food_info_en

In [7]:
food_en('비빔')

('비빔', '요리를 찾지 못했습니다.', '요리를 찾지 못했습니다.')

# 알레르기정보

In [8]:
def food_resource(name):
    failment = str('레시피를 찾을 수 없습니다.')
    try:
        rc_no = int(data2.loc[name]['RECIPE_ID'])
        resource = data3[data3.RECIPE_ID == rc_no]['IRDNT_NM'].tolist()

        result = resource
    except:
        result = failment
        
    try:
        alwar = set()
        for key, value in Allergie.items():
            alsource = set(value) & set(result)
            if alsource != set():
                alwar = alwar | alsource
            else:
                pass
    except:
        pass
    
    try:
        keys = []
        for key, value in Allergie.items():
            if alwar & set(value) != set():
                keys.append(key)
    except:
        pass
  
    if keys != []:
      return result, keys
    else:
      return result, keys

In [9]:
food_resource('비빔밥')

(['쌀',
  '안심',
  '콩나물',
  '청포묵',
  '미나리',
  '소금',
  '국간장',
  '다진파',
  '다진마늘',
  '참기름',
  '고추장',
  '설탕',
  '숙주',
  '도라지',
  '고사리',
  '계란',
  '양지머리'],
 ['대두', '계란', '쇠고기'])

# 음식정보[최종]

In [10]:
def food_info(name):
    food_en(name)
    food_resource(name)

In [11]:
food_info('비빔밥')

# 네이버 API 적용

In [12]:
! mkdir static

하위 디렉터리 또는 파일 static이(가) 이미 있습니다.


In [13]:
def foodname_voice(name):
  import requests
  import json
  import os
  import sys
  import urllib.request
  import shutil

  client_id = "fr2pmzwe08"
  client_secret = "uzTbsKj83c57qTJNlHRbeIsTiwDdVoZLKDod8UBh"
  encText = urllib.parse.quote(name)
  data = "speaker=nara&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
  url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
  request.add_header("X-NCP-APIGW-API-KEY",client_secret)
  response = urllib.request.urlopen(request, data=data.encode('utf-8'))
  rescode = response.getcode()
  if(rescode==200):
      response_body = response.read()
      with open('foodname.mp3', 'wb') as f:
          f.write(response_body)
      source = 'foodname.mp3'
      destination = 'static/foodname.mp3'
      shutil.move(source, destination)
  else:
      print("Error Code:" + rescode)

In [14]:
def kfoodinfo(name):
  food_en(name)
  food_resource(name)
  foodname_voice(name)

In [15]:
kfoodinfo('비빔밥')

**모듈 임포트**

In [22]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

In [16]:
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn

**템플릿 생성**

In [17]:
!mkdir templates

하위 디렉터리 또는 파일 templates이(가) 이미 있습니다.


In [25]:
%%writefile templates/input.html
<!DOCTYPE html>
<html>
<body>

<h2>HTML Forms</h2>

<form action="/topic" method='post'>
  <label for="fname">요리명:</label><br>
  <input type="text" id="fname" name="fname" value="요리명"><br><br>
  <input type="submit" value="Submit">
</form> 

<p>If you click the "Submit" button, the form-data will be sent to a page called "/action_page.php".</p>

</body>
</html>


Writing templates/input.html


In [65]:
%%writefile templates/result_video.html
<!DOCTYPE html>
<html>
<body>

<h1>찾으신 한식에 관련된 정보입니다.</h1>
<ul>

<li>요리명 : {{foodname}}</li>
<li>요리명(영문) : {{fooden}}</li>
<li>요리설명(영문) : {{foodinen}}</li>
<li>요리재료 : {{foodsource}}</li>

<script>
    if ({{allergie}}!=[] ) {
        <li>※경고 : 알러지 유발재료 {{allergie}} 가 포함되어 있을 수 있습니다.</li>
      } else {
        <li></li>
      }
</script>
    
</ul>

<p>음식명(한국어) 발음:</p>

<audio controls autoplay>
  <source src= {{ fn }} type="audio/mpeg">
  Your browser does not support the audio element.
</audio>

</body>
</html>


Overwriting templates/result_video.html


**API 구성 및 실행**

In [66]:
from fastapi.staticfiles import StaticFiles

app=FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount('/static', StaticFiles(directory='static'), name='static')

@app.get('/', response_class=HTMLResponse)
async def read_topic(request: Request):
  return templates.TemplateResponse("input.html", {"request": request})

@app.post('/topic', response_class=HTMLResponse)
async def get_topic(request: Request, fname: str = Form(...)):
  food_en(fname)
  food_resource(fname)
  foodname_voice(fname)

  return templates.TemplateResponse("result_video.html", {"request": request,
                                                          'fn': '/static/foodname.mp3',
                                                          'foodname': fname,
                                                          'fooden': food_en(fname)[1],
                                                          'foodinen' : food_en(fname)[2],
                                                          'foodsource' : food_resource(fname)[0],
                                                          'allergie' : food_resource(fname)[1]
                                                          })



In [67]:
!ngrok authtoken 26yWCLJ3Z9w72caEDhAo2a5x1Bd_37x6zHiyhnZ4LhYECiqAM

Authtoken saved to configuration file: C:\Users\kk135/.ngrok2/ngrok.yml


In [68]:
app.url_path_for('static', path='/foodname.mp3')

'/static/foodname.mp3'

In [69]:
str(food_resource('비빔밥')[0])

"['쌀', '안심', '콩나물', '청포묵', '미나리', '소금', '국간장', '다진파', '다진마늘', '참기름', '고추장', '설탕', '숙주', '도라지', '고사리', '계란', '양지머리']"

In [70]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://fd73-112-187-142-248.ngrok.io


INFO:     Started server process [14252]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     182.172.56.228:0 - "GET / HTTP/1.1" 200 OK
INFO:     112.187.142.248:0 - "GET / HTTP/1.1" 200 OK
INFO:     112.187.142.248:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     112.187.142.248:0 - "POST /topic HTTP/1.1" 200 OK
INFO:     112.187.142.248:0 - "GET /foodname.mp3 HTTP/1.1" 200 OK
INFO:     112.187.142.248:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     112.187.142.248:0 - "POST /topic HTTP/1.1" 200 OK
INFO:     112.187.142.248:0 - "GET /foodname.mp3 HTTP/1.1" 200 OK
INFO:     112.187.142.248:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14252]


In [80]:
food_resource('가지볶음')[0]

'레'